## About Dataset
Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis

#### Best Practises
1. Preprocessing And Cleaning
2. Train Test Split
3. BOW,TFIDF,Word2vec
4. Train ML algorithms

In [43]:
# Load the dataset
import pandas as pd
data=pd.read_csv('Kindle Reviews/all_kindle_review.csv')
# data.head(2)
df=data[['reviewText','rating']]          #extracting useful data from dataset
df.head()
# df.shape  #(12000, 2)

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [44]:
## Missing Values
df.isnull().sum()             #reviewText    0                     #rating    0
df['rating'].unique()           #array([3, 5, 4, 2, 1], dtype=int64)
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

### Preprocessing And Cleaning
x represents each individual value in the "rating" column.

apply() applies the lambda function to each value in the column.

In [45]:
## postive review is 1 and negative review is 0
df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)  
df['rating'].value_counts()


C:\Users\Abhinav\AppData\Local\Temp\ipykernel_53736\3380202468.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)


rating
1    8000
0    4000
Name: count, dtype: int64

#### .str: Applies string operations to each value in the column
If you don't write .str, the code will throw an error because Pandas applies .lower() only on string values inside a DataFrame using .str.

In [46]:
## 1. Lower All the cases
df['reviewText']=df['reviewText'].str.lower()
df.head()

C:\Users\Abhinav\AppData\Local\Temp\ipykernel_53736\4069475088.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].str.lower()


,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [47]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from bs4 import BeautifulSoup  #library to remove html tags

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abhinav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### 1
re.sub(pattern, replacement, string)
x -> string: The input string where replacements will be made.

- → Allows the hyphen (-).

]+ →The + means "one or more occurrences" of the preceding set ([^...]).
This ensures that multiple unwanted characters are removed together.

#### 2
This list comprehension iterates over every word (y) in the review.
It keeps only the words that are NOT in stopwords.words('english')

2.phle reviewtext ko words m split krenge , fir stopwords remove krke , wapas sentence bna denge
#### 3
r'...' is a raw string literal in Python.
In regular strings, backslashes (\) are escape characters (e.g., \n for newline, \t for tab).
Raw strings (r'...') tell Python to ignore escape sequences, so the pattern remains unchanged.

re.sub is only for string


In [48]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))


C:\Users\Abhinav\AppData\Local\Temp\ipykernel_53736\1658462958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
C:\Users\Abhinav\AppData\Local\Temp\ipykernel_53736\1658462958.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
C:\Users\Abhinav\AppData\Local\Temp\ipykernel_53736\1658462958.py:6: SettingW

In [49]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [67]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))
df.head()




C:\Users\Abhinav\AppData\Local\Temp\ipykernel_53736\2708133550.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))


,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [72]:
import gensim
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

from nltk import sent_tokenize
from gensim.utils import simple_preprocess
words=[]
for sent in df['reviewText']:
    sent_token=sent_tokenize(sent)   #Tokenizes the sentence into smaller sentences using sent_tokenize().
    for sent in sent_token:                   #Iterates through each split sentence 
        words.append(simple_preprocess(sent))
words

model=gensim.models.Word2Vec(words)    
model.wv.index_to_key 

import numpy as np
def avg_word2vec(doc):
    valid_words = [word for word in doc if word in model.wv.index_to_key]
    if len(valid_words) == 0:  
        return np.zeros(100)  # Return a 100D zero vector for empty cases
    return np.mean([model.wv[word] for word in valid_words], axis=0)

from tqdm import tqdm
X=[]                                    
for i in tqdm(range(len(words))):  
    X.append(avg_word2vec(words[i]))
len(X)                                  #5569 tokenized sentences
X 
d=pd.DataFrame()                             #reshaping all arrays                                     
    
d = pd.concat([pd.DataFrame(X[i].reshape(1, -1)) for i in range(len(X))], ignore_index=True)
y = df['rating']
d.head()


KeyError: 'reviewText'

When using CountVectorizer or tfidf the output is a sparse matrix by default.
Calling .toarray() converts this sparse matrix into a dense NumPy array.



In [ ]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],
                                              test_size=0.20)

from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

# X_train_bow


naive_bayes bcoz it works directly on sparse matrix

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
y_pred_bow=nb_model_bow.predict(X_test_bow)
y_pred_tfidf=nb_model_bow.predict(X_test_tfidf)

confusion_matrix(y_test,y_pred_bow)





In [63]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.5641666666666667


In [64]:
confusion_matrix(y_test,y_pred_tfidf)

array([[508, 302],
       [745, 845]], dtype=int64)

In [65]:
print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.56375
